In [1]:
import os
import sys
import glob
import torch
import numpy as np
import pandas as pd
import open3d as o3d

In [2]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir, "data")
src_dir = os.path.join(base_dir, "src")
sys.path.append(os.path.join(src_dir))

In [3]:
train_files = glob.glob(os.path.join(data_dir, "original", "train", "*", "*.obj"))
valid_files = glob.glob(os.path.join(data_dir, "original", "val", "*", "*.obj"))
len(train_files), len(valid_files)

(7003, 1088)

In [4]:
from utils import load_pipeline

In [13]:
class Tokenizer(object):
    
    def _padding(self, ids_tensor, pad_token):
        max_length = max([len(ids) for ids in ids_tensor])
        ids_tensor = [
            torch.cat([
                ids, pad_token.repeat(max_length - len(ids) + 1)
            ])
            for ids in ids_tensor
        ]
        return ids_tensor
    
    def _make_position_ids_tensor(self, ids_tensor):
        bsz, length = ids_tensor.shape
        position_ids = torch.arange(length)[None, :]
        position_ids = position_ids.repeat(bsz, 1)
        return position_ids
    
    def _make_position_ids_list(self, ids_list):
        bsz = len(ids_list)
        position_ids = [torch.arange(len(ids)) for ids in ids_list]
        return position_ids
    
    def _make_padding_mask(self, ids_tensor, pad_id):
        mask = torch.where(
            ids_tensor==pad_id,
            torch.ones_like(ids_tensor),
            torch.zeros_like(ids_tensor)
        ).type(torch.bool)
        return mask

    def _make_future_mask(self, ids_tensor):
        batch, length = ids_tensor.shape
        arange = torch.arange(length)
        mask = torch.where(
            arange[None, :] <= arange[:, None],
            torch.zeros((length, length)),
            torch.ones((length, length))*(-np.inf)
        ).type(torch.float32)
        return mask

In [32]:
class EncodeVertexTokenizer(Tokenizer):
    
    def __init__(self, pad_id=0):
        self.pad_token = torch.tensor([pad_id])
        self.pad_id = pad_id
        
    def tokenize(self, vertices, padding=True):
        vertices = [v.reshape(-1,) + 1 for v in vertices]
        coord_type_tokens = [torch.arange(len(v)) % 3 + 1 for v in vertices]
        
        if padding:
            vertices = torch.stack(self._padding(vertices, self.pad_token))
            coord_type_tokens = torch.stack(self._padding(coord_type_tokens, self.pad_token))
            padding_mask = self._make_padding_mask(vertices, self.pad_id)
            position_ids = self._make_position_ids_tensor(vertices)
            outputs = {
                "value_tokens": vertices,
                "coord_type_tokens": coord_type_tokens,
                "position_ids": position_ids,
                "padding_mask": padding_mask,
            }
        else:
            position_ids = self._make_position_ids_list(vertices)
            outputs = {
                "value_token": vertices,
                "coord_type_tokens": coord_type_tokens,
                "position_ids": position_ids,
            }
            
        return outputs

In [33]:
class DecodeVertexTokenizer(Tokenizer):
    
    def __init__(self, bos_id=0, eos_id=1, pad_id=2):
        
        self.special_tokens = {
            "bos": torch.tensor([bos_id]),
            "eos": torch.tensor([eos_id]),
            "pad": torch.tensor([pad_id]),
        }
        self.pad_id = pad_id
        self.not_coord_token = torch.tensor([0])
    
    def tokenize(self, vertices, padding=True):
        special_tokens = self.special_tokens
        not_coord_token = self.not_coord_token
        
        vertices = [
            torch.cat([
                special_tokens["bos"], 
                v.reshape(-1,)  + len(special_tokens), 
                special_tokens["eos"]
            ])
            for v in vertices
        ]
        
        coord_type_tokens = [
            torch.cat([
                not_coord_token,
                torch.arange(len(v)) % 3 + 1,
                not_coord_token
            ])
            for v in vertices
        ]
        
        if padding:
            vertices = torch.stack(self._padding(vertices, special_tokens["pad"]))
            coord_type_tokens = torch.stack(self._padding(coord_type_tokens, not_coord_token))
            padding_mask = self._make_padding_mask(vertices, self.pad_id)
            future_mask = self._make_future_mask(vertices)
            position_ids = self._make_position_ids_tensor(vertices)
            outputs = {
                "value_tokens": vertices,
                "coord_type_tokens": coord_type_tokens,
                "position_ids": position_ids,
                "padding_mask": padding_mask,
                "future_mask": future_mask,
            }
        else:
            position_ids = self._make_position_ids_list(vertices)
            outputs = {
                "value_tokens": vertices,
                "coord_type_tokens": coord_type_tokens,
                "position_ids": position_ids,
            }
            
        return outputs
    
    def detokenize(self, vertices):
        special_tokens = self.special_tokens
        
        result = []
        for vertex in vertices:
            vertex = vertex - len(special_tokens)
            result.append(
                vertex[torch.where(vertex >= 0)]
            )
        return result

In [8]:
class FaceTokenizer(Tokenizer):
    
    def __init__(self, eof_id=0, eos_id=1, pad_id=2):
        self.special_tokens = {
            "eof": torch.tensor([eof_id]),
            "eos": torch.tensor([eos_id]),
            "pad": torch.tensor([pad_id]),
        }
        self.pad_id = pad_id
        
    def tokenize(self, faces, target=False, padding=True):
        special_tokens = self.special_tokens
        faces = [
            torch.cat([
                torch.cat([
                    f + len(special_tokens), special_tokens["eof"].repeat(len(f))[:, None]], dim=1
                ).reshape(-1, ),
                special_tokens["eos"]
            ]) for f in faces
        ]
        
        if padding:
            faces = self._padding(faces, special_tokens["pad"])
            
            if target:
                faces = [torch.cat([f, special_tokens["pad"]])[1:] for f in faces]
            
            faces = torch.stack(faces)
            pad_mask = self._make_padding_mask(faces, self.pad_id)
            future_mask = self._make_future_mask(faces)
            return faces, pad_mask, future_mask
        else:
            return faces

    def detokenize(self, faces):
        special_tokens = self.special_tokens
        
        result = []
        for face in faces:
            face = face - len(special_tokens)
            result.append(
                face[torch.where(face >= 0)]
            )
        return result

In [15]:
v_batch, f_batch = [], []
for i in range(3):
    v, f = load_pipeline(train_files[i])
    v = torch.tensor(v)
    f = torch.tensor(f)
    v_batch.append(v)
    f_batch.append(f)
    print(v.shape, f.shape)

torch.Size([677, 3]) torch.Size([1317, 3])
torch.Size([342, 3]) torch.Size([676, 3])
torch.Size([648, 3]) torch.Size([1184, 3])


In [34]:
enc_vtk = EncodeVertexTokenizer()
dec_vtk = DecodeVertexTokenizer()
ftk = FaceTokenizer()

In [35]:
enc_vtk.tokenize(v_batch)

{'value_tokens': tensor([[161, 137, 131,  ..., 137, 125,   0],
         [135, 134, 128,  ...,   0,   0,   0],
         [163,  99, 130,  ...,   0,   0,   0]]),
 'coord_type_tokens': tensor([[1, 2, 3,  ..., 2, 3, 0],
         [1, 2, 3,  ..., 0, 0, 0],
         [1, 2, 3,  ..., 0, 0, 0]]),
 'position_ids': tensor([[   0,    1,    2,  ..., 2029, 2030, 2031],
         [   0,    1,    2,  ..., 2029, 2030, 2031],
         [   0,    1,    2,  ..., 2029, 2030, 2031]]),
 'padding_mask': tensor([[False, False, False,  ..., False, False,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True]])}

In [36]:
enc_vtk.tokenize(v_batch, padding=False)

{'value_token': [tensor([161, 137, 131,  ...,  95, 137, 125], dtype=torch.int32),
  tensor([135, 134, 128,  ..., 121,  65, 128], dtype=torch.int32),
  tensor([163,  99, 130,  ...,  93,  88, 126], dtype=torch.int32)],
 'coord_type_tokens': [tensor([1, 2, 3,  ..., 1, 2, 3]),
  tensor([1, 2, 3,  ..., 1, 2, 3]),
  tensor([1, 2, 3,  ..., 1, 2, 3])],
 'position_ids': [tensor([   0,    1,    2,  ..., 2028, 2029, 2030]),
  tensor([   0,    1,    2,  ..., 1023, 1024, 1025]),
  tensor([   0,    1,    2,  ..., 1941, 1942, 1943])]}

In [37]:
[v.reshape(-1, ) for v in v_batch]

[tensor([160, 136, 130,  ...,  94, 136, 124], dtype=torch.int32),
 tensor([134, 133, 127,  ..., 120,  64, 127], dtype=torch.int32),
 tensor([162,  98, 129,  ...,  92,  87, 125], dtype=torch.int32)]

In [38]:
dec_vtk.tokenize(v_batch)

{'value_tokens': tensor([[  0, 163, 139,  ..., 127,   1,   2],
         [  0, 137, 136,  ...,   2,   2,   2],
         [  0, 165, 101,  ...,   2,   2,   2]]),
 'coord_type_tokens': tensor([[0, 1, 2,  ..., 2, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0],
         [0, 1, 2,  ..., 0, 0, 0]]),
 'position_ids': tensor([[   0,    1,    2,  ..., 2031, 2032, 2033],
         [   0,    1,    2,  ..., 2031, 2032, 2033],
         [   0,    1,    2,  ..., 2031, 2032, 2033]]),
 'padding_mask': tensor([[False, False, False,  ..., False, False,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True]]),
 'future_mask': tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
         [0., 0., -inf,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         ...,
         [0., 0., 0.,  ..., 0., -inf, -inf],
         [0., 0., 0.,  ..., 0., 0., -inf],
         [0., 0., 0.,  ..., 0., 0., 0.]])}

In [39]:
dec_vtk.tokenize(v_batch, padding=False)

{'value_tokens': [tensor([  0, 163, 139,  ..., 139, 127,   1]),
  tensor([  0, 137, 136,  ...,  67, 130,   1]),
  tensor([  0, 165, 101,  ...,  90, 128,   1])],
 'coord_type_tokens': [tensor([0, 1, 2,  ..., 1, 2, 0]),
  tensor([0, 1, 2,  ..., 1, 2, 0]),
  tensor([0, 1, 2,  ..., 1, 2, 0])],
 'position_ids': [tensor([   0,    1,    2,  ..., 2030, 2031, 2032]),
  tensor([   0,    1,    2,  ..., 1025, 1026, 1027]),
  tensor([   0,    1,    2,  ..., 1943, 1944, 1945])]}

In [40]:
[v.reshape(-1, ) for v in v_batch]

[tensor([160, 136, 130,  ...,  94, 136, 124], dtype=torch.int32),
 tensor([134, 133, 127,  ..., 120,  64, 127], dtype=torch.int32),
 tensor([162,  98, 129,  ...,  92,  87, 125], dtype=torch.int32)]

In [41]:
outputs = dec_vtk.detokenize(dec_vtk.tokenize(v_batch)["value_tokens"])
outputs

[tensor([160, 136, 130,  ...,  94, 136, 124]),
 tensor([134, 133, 127,  ..., 120,  64, 127]),
 tensor([162,  98, 129,  ...,  92,  87, 125])]

In [17]:
fs, m_pad, m_future = ftk.tokenize(f_batch)
print(fs.shape)
print(fs)
print("="*60)
print(m_pad.shape)
print(m_pad)
print("="*60)
print(m_future.shape)
print(m_future)

torch.Size([3, 5270])
tensor([[  3,   4, 174,  ...,   0,   1,   2],
        [  3,   6,  10,  ...,   2,   2,   2],
        [  3,   4,   7,  ...,   2,   2,   2]])
torch.Size([3, 5270])
tensor([[False, False, False,  ..., False, False,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])
torch.Size([5270, 5270])
tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [18]:
outputs = ftk.detokenize(fs)
outputs

[tensor([  0,   1, 171,  ..., 672, 675, 676]),
 tensor([  0,   3,   7,  ..., 335, 340, 341]),
 tensor([  0,   1,   4,  ..., 642, 646, 647])]

In [21]:
fs, _, _ = ftk.tokenize(f_batch, target=True)
print(fs.shape)
print(fs)
print("="*60)

torch.Size([3, 5270])
tensor([[  4, 174,   0,  ...,   1,   2,   2],
        [  6,  10,   0,  ...,   2,   2,   2],
        [  4,   7,   0,  ...,   2,   2,   2]])


In [20]:
outputs = ftk.detokenize(fs)
outputs

[tensor([  1, 171,   0,  ..., 672, 675, 676]),
 tensor([  3,   7,   0,  ..., 335, 340, 341]),
 tensor([  1,   4,   0,  ..., 642, 646, 647])]